In [1]:
# =========================
# X - IMPORTS
# =========================

from X_functions import *
from X_config import *
from X_models import *
from X_interpolation import *
from X_plots import *

# =========================


c:\Users\sal_f\anaconda3\envs\env_ggs0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = GGS_config_static(date=dt.datetime.now(timezone.utc))
root_directory = GGS_config_output(config, path="default")
datetime_index = "2024-02-18 00:00:00.000000+00:00"




### Glider Guidance System (GGS) Configuration ###

Glider name: Yucatan

Execution date: 2024-02-18 21:58:44.004914+00:00

Max Depth: 100

Extent:
  Boundary 1: (15, -93)
  Boundary 2: (30, -77)

GPS_coords:
  GPS_coord 1: (39.4, -74.2)
  GPS_coord 2: (39.3, -71.2)



In [3]:
rtofs = RTOFS()
rtofs.rtofs_load(datetime_index)
rtofs.rtofs_subset(config, subset=True) # [subset] OPTIONS: True = Config Extent, False = Full Extent
rtofs.rtofs_save(config, root_directory, save_data=False, save_qc=False) # [save] OPTIONS: True = Save Data File, False = Do Not Save Data File
rtofs_data = rtofs.data
rtofs_qc = rtofs.qc

depth_average_rtofs, bin_average_rtofs = interpolate_rtofs(config, root_directory, rtofs_data, chunk=False, save_depth_average=True, save_bin_average=False) # [save] OPTIONS: True = Save Data File, False = Do Not Save Data File

# depth_average_rtofs = xr.open_dataset(root_directory + "/" + "Yucatan_RTOFS_DepthAverage_20240218T00Z.nc")


### MODEL DATA: [RTOFS] ###

MODEL DATA ACQUIRED
Requested datetime: 2024-02-18 00:00:00.000000+00:00
Nearest datetime index in the dataset: 6642
Acquired datetime: 2024-02-18T00:00:00.000000000

### INTERPOLATING MODEL DATA ###

Start time (UTC): 2024-02-18 21:58:52.536516
End time (UTC): 2024-02-18 22:02:21.309572
Run time: 0:03:28.773056


In [4]:
cmems = CMEMS(username='sfricano1', password='GlobalGliders1')
cmems.cmems_load(config, datetime_index)
cmems.cmems_subset(config, subset=True) # [subset] OPTIONS: True = Depth Subset, False = Full Depth Range
cmems.cmems_save(config, root_directory, save_data=False, save_qc=False) # [save] OPTIONS: True = Save Data File, False = Do Not Save Data File
cmems_data = cmems.data
cmems_qc = cmems.qc

depth_average_cmems, bin_average_cmems = interpolate_cmems(config, root_directory, cmems_data, chunk=False, save_depth_average=True, save_bin_average=False)

# depth_average_cmems = xr.open_dataset(root_directory + "/" + "Yucatan_CMEMS_DepthAverage_20240218T00Z.nc")


### MODEL DATA: [CMEMS] ###

INFO - 2024-02-18T22:02:21Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-02-18T22:02:21Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-02-18T22:02:23Z - Service was not specified, the default one was selected: "arco-geo-series"

### INTERPOLATING MODEL DATA ###

Start time (UTC): 2024-02-18 22:02:26.472398
End time (UTC): 2024-02-18 22:05:12.399347
Run time: 0:02:45.926949


In [5]:
latitude_qc = '20.30'
longitude_qc = '-86.50'

GGS_plot_magnitude(config, root_directory, depth_average_rtofs, latitude_qc, longitude_qc, model_name="rtofs", density=2, show_gliders=True, show_route=False, show_qc=False, manual_extent=None)
GGS_plot_threshold(config, root_directory, depth_average_rtofs, latitude_qc, longitude_qc, model_name="rtofs", density=2, mag1=0.0, mag2=0.2, mag3=0.3, mag4=0.4, mag5=0.5, show_gliders=True, show_route=False, show_qc=False, manual_extent=None)

GGS_plot_magnitude(config, root_directory, depth_average_cmems, latitude_qc, longitude_qc, model_name="cmems", density=2, show_gliders=True, show_route=False, show_qc=False, manual_extent=None)
GGS_plot_threshold(config, root_directory, depth_average_cmems, latitude_qc, longitude_qc, model_name="cmems", density=2, mag1=0.0, mag2=0.2, mag3=0.3, mag4=0.4, mag5=0.5, show_gliders=True, show_route=False, show_qc=False, manual_extent=None)



### CREATING RTOFS MAGNITUDE PLOT ###

Start time (UTC): 2024-02-18 22:05:12.408054
Found 7 Glider Datasets within search window.
Glider Indexes found: sp071-20231220T1541, sg625-20240119T0000, sg652-20240119T0000, sp065-20240214T1520, mote-dora-20240118T0000, stommel-20240128T1522, ru38-20240128T1523
End time (UTC): 2024-02-18 22:06:52.496787
Run time: 0:01:40.088733

### CREATING RTOFS THRESHOLD PLOT ###

Start time (UTC): 2024-02-18 22:06:52.498800
Found 7 Glider Datasets within search window.
Glider Indexes found: sp071-20231220T1541, sg625-20240119T0000, sg652-20240119T0000, sp065-20240214T1520, mote-dora-20240118T0000, stommel-20240128T1522, ru38-20240128T1523
End time (UTC): 2024-02-18 22:07:15.051888
Run time: 0:00:22.553088

### CREATING CMEMS MAGNITUDE PLOT ###

Start time (UTC): 2024-02-18 22:07:15.054389
Found 7 Glider Datasets within search window.
Glider Indexes found: sp071-20231220T1541, sg625-20240119T0000, sg652-20240119T0000, sp065-20240214T1520, mote-dora-20240118T

In [9]:
### FUNCTION:
def GGS_plot_magnitude_dual(config, directory, depth_average_data_1, depth_average_data_2, latitude_qc, longitude_qc, model_name_1=None, model_name_2=None, density=2, show_gliders=False, show_route=False, show_qc=False, manual_extent=None):
    
    '''
    Plot the depth-averaged current fields from two datasets side by side.

    Args:
    - config (dict): Glider Guidance System mission configuration.
    - directory (str): Glider Guidance System mission directory.
    - depth_average_data1, depth_average_data2 (xarray.Dataset): Datasets with the computed variables and layer information for side-by-side comparison.
    - qc_latitude (float): Latitude of the QC sample point.
    - qc_longitude (float): Longitude of the QC sample point.
    - model_name (str, optional): Name of the model.
    - density (int, optional): Density of the streamplot.
    - show_route (bool, optional): Show the route on the plot.
    - show_qc (bool, optional): Show the QC sample point.
    - manual_extent (list or str, optional): Manual specification of plot extent as [west, east, south, north], or "config" to use extent from config.

    Returns:
    - None
    '''

    # INITIALIZATION
    if model_name_1 is None:
        model_name_1 = "Unknown"
    else:
        model_name_1 = model_name_1.upper()
    if model_name_2 is None:
        model_name_2 = "Unknown"
    else:
        model_name_2 = model_name_2.upper()
    
    print(f"\n### CREATING DUAL ({model_name_1}, {model_name_2}) MAGNITUDE PLOT ###\n")
    start_time = dt.datetime.now()

    # PLOTTING FUNCTION
    def plot_magnitude(ax, config, depth_average_data, latitude_qc, longitude_qc, model_name, density, show_gliders, show_route, show_qc, manual_extent):
        
        # DATA EXTRACTION
        longitude = depth_average_data.lon.values.squeeze()
        latitude = depth_average_data.lat.values.squeeze()
        u_depth_avg = depth_average_data['u_depth_avg'].values.squeeze()
        v_depth_avg = depth_average_data['v_depth_avg'].values.squeeze()
        mag_depth_avg = depth_average_data['mag_depth_avg'].values.squeeze()
        
        # EXTENT SETUP
        if manual_extent == "config":
            map_extent = [config['extent'][0][1], config['extent'][1][1], config['extent'][0][0], config['extent'][1][0]]
        elif isinstance(manual_extent, list) and len(manual_extent) == 4:
            map_extent = manual_extent
        else:
            data_extent_lon = [float(longitude.min()), float(longitude.max())]
            data_extent_lat = [float(latitude.min()), float(latitude.max())]
            map_extent = data_extent_lon + data_extent_lat
        ax.set_extent(map_extent, crs=ccrs.PlateCarree())
        plot_formatted_ticks(ax, map_extent[:2], map_extent[2:], proj=ccrs.PlateCarree(), fontsize=10, label_left=True, label_right=False, label_bottom=True, label_top=False, gridlines=True)

        # PLOT ELEMENTS
        levels, ticks, extend = plot_contour_cbar(mag_depth_avg, max_levels=10, extend_max=True)
        contourf = ax.contourf(longitude, latitude, mag_depth_avg, levels=levels, cmap=cmo.speed, transform=ccrs.PlateCarree(), zorder=10, extend=extend)
        streamplot = ax.streamplot(longitude, latitude, u_depth_avg, v_depth_avg, density=density, linewidth=0.5, color='black', transform=ccrs.PlateCarree(), zorder=10)

        # GLIDERS
        if show_gliders:
            glider_dataframes = acquire_gliders(extent=map_extent, target_date=dt.datetime.now(), date_delta=dt.timedelta(days=1), requested_variables=["time", "longitude", "latitude", "profile_id", "depth"], print_vars=False, target="all", request_timeout=5, enable_parallel=False)
            plot_add_gliders(ax, glider_dataframes, legend=True)
            glider_legend = ax.get_legend()
            if glider_legend:
                glider_legend.get_frame().set_alpha(0.5)
                ax.add_artist(glider_legend)
        
        # ROUTE
        if show_route:
            lats, lons = zip(*config["GPS_coords"])
            ax.plot(lons, lats, 'w-', transform=ccrs.PlateCarree(), linewidth=2.5, zorder=91)
            ax.plot(lons, lats, 'k', transform=ccrs.PlateCarree(), linewidth=1.0, linestyle='--', alpha=0.6, zorder=92)
            
            start_coords = config["GPS_coords"][0]
            end_coords = config["GPS_coords"][-1]
            ax.scatter(*start_coords[::-1], color='purple', s=100, transform=ccrs.PlateCarree(), zorder=93)
            for GPS_coord in config["GPS_coords"][1:-1]:
                ax.scatter(*GPS_coord[::-1], color='purple', s=100, transform=ccrs.PlateCarree(), zorder=93)
            ax.scatter(*end_coords[::-1], color='purple', s=100, transform=ccrs.PlateCarree(), zorder=93)
        
        # QUALITY CONTROL
        if show_qc:
            (y_index, x_index), (lat_index, lon_index) = calculate_gridpoint(depth_average_data, latitude_qc, longitude_qc)
            qc_lon = depth_average_data['lon'].isel(x=x_index, y=y_index).values
            qc_lat = depth_average_data['lat'].isel(x=x_index, y=y_index).values
            circle = Circle((qc_lon, qc_lat), radius=0.25, edgecolor='purple', facecolor='none', linewidth=2, transform=ccrs.PlateCarree(), zorder=95)
            ax.add_patch(circle)
        
        # FEATURES
        ax.add_feature(cfeature.GSHHSFeature(scale='full'), edgecolor="black", facecolor="tan", linewidth=0.25, zorder=90)
        ax.add_feature(cfeature.RIVERS, edgecolor="steelblue", linewidth=0.25, zorder=90)
        ax.add_feature(cfeature.LAKES, edgecolor="black", facecolor="lightsteelblue", linewidth=0.25, zorder=90)
        ax.add_feature(cfeature.BORDERS, edgecolor="black", linewidth=0.25, zorder=90)
        
        plot_bathymetry(ax, config, depth_average_data, isobath1=-100, isobath2=-1000, show_legend=False)
        bathymetry_legend = ax.get_legend()
        if bathymetry_legend:
            bathymetry_legend.get_frame().set_alpha(0.5)
            ax.add_artist(bathymetry_legend)

        # COLORBAR
        cbar = fig.colorbar(contourf, orientation='vertical', extend=extend)
        format_colorbar(fig, ax, cbar)
        cbar.set_label('Depth Averaged Current Magnitude (m/s)', labelpad=10)
        cbar.set_ticks(ticks)
        cbar.set_ticklabels([f"{tick:.1f}" for tick in ticks])

    # PLOT SETUP
    fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(20, 10))

    # PLOT EXECUTION
    plot_magnitude(ax1, config, depth_average_data_1, latitude_qc, longitude_qc, model_name_1, density, show_gliders, show_route, show_qc, manual_extent)
    plot_magnitude(ax2, config, depth_average_data_2, latitude_qc, longitude_qc, model_name_2, density, show_gliders, show_route, show_qc, manual_extent)

    # TITLES
    ax1.set_title(f"{model_name_1}", fontsize=14, fontweight='bold', pad=20)
    ax2.set_title(f"{model_name_2}", fontsize=14, fontweight='bold', pad=20)
    title_text = f"Depth Averaged Currents - Depth Range: {config['max_depth']}m"
    model_name_dual = f"{model_name_1} vs. {model_name_2}"
    format_titles(ax1, fig, config, model_data=depth_average_data_1, model_name=model_name_dual, title=title_text)
    
    # SAVE & CLOSE
    file_datetime = format_save_datetime(depth_average_data_1)
    fig_filename = f"GGS_{config['glider_name']}_Magnitude_Dual_{config['max_depth']}m_{file_datetime}.png"
    fig_path = os.path.join(directory, fig_filename)
    fig.savefig(fig_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    # LOGGING
    end_time = dt.datetime.now()
    print(f"Runtime: {end_time - start_time}")


In [10]:
GGS_plot_magnitude_dual(config, root_directory, depth_average_rtofs, depth_average_cmems, latitude_qc, longitude_qc, model_name_1="rtofs", model_name_2="cmems", density=2, show_gliders=True, show_route=False, show_qc=False, manual_extent=[-88, -84, 18, 22])



### CREATING DUAL (RTOFS, CMEMS) MAGNITUDE PLOT ###

Found 4 Glider Datasets within search window.
Glider Indexes found: sg625-20240119T0000, sg652-20240119T0000, stommel-20240128T1522, ru38-20240128T1523
Found 4 Glider Datasets within search window.
Glider Indexes found: sg625-20240119T0000, sg652-20240119T0000, stommel-20240128T1522, ru38-20240128T1523
Runtime: 0:00:41.489332
